In [1]:
!pip install openai

In [2]:
from openai import AzureOpenAI

from google.colab import userdata

AZURE_ENDPOINT = userdata.get('Azure_Endpoint')
API_KEY = userdata.get('Azure_Key')
API_VERSION = userdata.get('Azure_Version')
MODEL = userdata.get('Model')

# Initialize OpenAI client
client = AzureOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=API_KEY,
    api_version=API_VERSION
)

In [3]:


response = client.chat.completions.create(
            model = "gpt-4o",
            messages= [{
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n    \n    CREATE TABLE Orders (\n      OrderID int,\n      CustomerID int,\n      OrderDate datetime,\n      OrderTime varchar(8),\n      PRIMARY KEY (OrderID)\n    );\n    \n    CREATE TABLE OrderDetails (\n      OrderDetailID int,\n      OrderID int,\n      ProductID int,\n      Quantity int,\n      PRIMARY KEY (OrderDetailID)\n    );\n    \n    CREATE TABLE Products (\n      ProductID int,\n      ProductName varchar(50),\n      Category varchar(50),\n      UnitPrice decimal(10, 2),\n      Stock int,\n      PRIMARY KEY (ProductID)\n    );\n    \n    CREATE TABLE Customers (\n      CustomerID int,\n      FirstName varchar(50),\n      LastName varchar(50),\n      Email varchar(100),\n      Phone varchar(20),\n      PRIMARY KEY (CustomerID)\n    );"
    },
    {
      "role": "user",
      "content": "Write a SQL query which computes the average total order value for all orders on 2023-04-01."
    }],
            temperature=0,
            max_tokens=900
        )

In [4]:
print(response.choices[0].message.content)

To compute the average total order value for all orders on 2023-04-01, you can use the following SQL query:

```sql
SELECT AVG(TotalOrderValue) AS AverageOrderValue
FROM (
    SELECT o.OrderID, SUM(od.Quantity * p.UnitPrice) AS TotalOrderValue
    FROM Orders o
    JOIN OrderDetails od ON o.OrderID = od.OrderID
    JOIN Products p ON od.ProductID = p.ProductID
    WHERE o.OrderDate = '2023-04-01'
    GROUP BY o.OrderID
) AS OrderValues;
```

This query works as follows:
1. It joins the `Orders`, `OrderDetails`, and `Products` tables to get the necessary information about each order.
2. It filters the orders to only include those made on '2023-04-01'.
3. It calculates the total value of each order by summing the product of `Quantity` and `UnitPrice` for each product in the order.
4. It computes the average of these total order values.
